
 ## Install Java, Spark, and Findspark



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
!tar xf spark-2.3.2-bin-hadoop2.7.tgz
!pip install -q findspark

## Set Environmental Variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"

## Find Spark and start session

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("stu_stop").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/food_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food_reviews.csv"), sep=",", header=True)
df.show()

+--------------------+
|             Reviews|
+--------------------+
|The pasta was a d...|
|We ate the fish i...|
|My family did not...|
|The girl even tri...|
|this is his job a...|
|I'm always greete...|
+--------------------+



In [ ]:
# Tokenize DataFrame
review_data = Tokenizer(inputCol="Reviews", outputCol="Words")

In [ ]:
# Transform DataFrame
reviewed = review_data.transform(df)
reviewed.show()

+--------------------+--------------------+
|             Reviews|               Words|
+--------------------+--------------------+
|The pasta was a d...|[the, pasta, was,...|
|We ate the fish i...|[we, ate, the, fi...|
|My family did not...|[my, family, did,...|
|The girl even tri...|[the, girl, even,...|
|this is his job a...|[this, is, his, j...|
|I'm always greete...|[i'm, always, gre...|
+--------------------+--------------------+



In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol="Words", outputCol="filtered")

In [ ]:
# Transform new DataFrame
newFrame = remover.transform(reviewed)
newFrame.show()

+--------------------+--------------------+--------------------+
|             Reviews|               Words|            filtered|
+--------------------+--------------------+--------------------+
|The pasta was a d...|[the, pasta, was,...|       [pasta, dish]|
|We ate the fish i...|[we, ate, the, fi...|  [ate, fish, tasty]|
|My family did not...|[my, family, did,...|[family, like, food]|
|The girl even tri...|[the, girl, even,...|[girl, even, trie...|
|this is his job a...|[this, is, his, j...|[job, since, prob...|
|I'm always greete...|[i'm, always, gre...|[always, greeted,...|
+--------------------+--------------------+--------------------+



In [ ]:
# Show simplified review
newFrame.select("filtered").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|filtered                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[pasta, dish]                                                                                                                                                                |
|[ate, fish, tasty]                                                                                                                                                           |
|[family, like, food]                                                                                                   